## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=16, kernel_size=(3,3), padding='same', input_shape=(32,32,3), activation='relu')) #32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.5))

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.5))

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.5))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=1000, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train, batch_size=100, epochs=100, validation_split=0.2, verbose=2)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Train on 40000 samples, validate on 10000 samples
Epoch 1/100
 - 10s - loss: 1.8096 - acc: 0.3697 - val_loss: 1.5314 - val_acc: 0.4505
Epoch 2/100
 - 7s - loss: 1.3930 - acc: 0.4943 - val_loss: 1.3357 - val_acc: 0.5289
Epoch 3/100
 - 7s - loss: 1.2238 - acc: 0.5576 - val_loss: 1.1080 - val_acc: 0.6097
Epoch 4/100
 - 7s - loss: 1.1046 - acc: 0.6019 - val_loss: 0.9912 - val_acc: 0.6567
Epoch 5/100
 - 7s - loss: 1.0240 - acc: 0.6359 - val_loss: 1.0368 - val_acc: 0.6506
Epoch 6/100
 - 7s - loss: 0.9573 - acc: 0.6600 - val_loss: 0.8802 - val_acc: 0.6912
Epoch 7/100
 - 7s - loss: 0.9092 - acc: 0.6760 - val_loss: 0.8517 - val_acc: 0.7055
Epoch 8/100
 - 7s - loss: 0.8627 - acc: 0.6958 - val_loss: 0.7618 - val_acc: 0.7316
Epoch 9/100
 - 7s - loss: 0.8271 - acc: 0.7089 - val_loss: 0.7707 - val_acc: 0.7302
Epoch 10/100
 - 7s - loss: 0.7883 - acc: 0.7219 - val_loss: 0.7025 - val_acc: 0.7556
Epoch 11/100
 - 7s - loss: 0.7516 - acc: 0.7336 - val_loss: 0.7208 - val_acc: 0.7520
Epoch 12/100
 - 7s - 

In [5]:
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 119us/step
Test loss: 0.6901617149829864
Test accuracy: 0.8058


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.94974357e-01, 1.14931034e-07, 4.22470155e-04, 4.69056278e-01,
        3.69026996e-02, 1.80225959e-10, 1.24107877e-10, 1.38958376e-11,
        1.98644161e-01, 2.57241619e-14]], dtype=float32)